In [1]:
from tensorflow.keras.models import load_model
model = load_model('/kaggle/input/mri-predictor/tensorflow2/f10.96/1/my_cnn_model.h5')


2024-04-24 01:59:43.768246: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 01:59:43.768337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 01:59:43.882765: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import os
import cv2
import numpy as np

def jpg_to_npy_in_folders(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, item in enumerate(os.listdir(input_folder)):
        item_path = os.path.join(input_folder, item)
        if os.path.isdir(item_path):
            jpg_to_npy_in_folders(item_path, output_folder)
        elif os.path.isfile(item_path) and item.endswith(".jpg"):
            img = cv2.imread(item_path)
            img = cv2.resize(img, (256, 256)) 
            img_array = np.array(img)
            folder = input_folder.split("/")[-1]
            output_file_path = os.path.join(output_folder, f"{folder}{i}.npy")
            np.save(output_file_path, img_array)

jpg_to_npy_in_folders("/kaggle/input/alzheimersdisease5classdatasetadni/Alzheimers-ADNI/train", "/kaggle/working/train")

In [3]:
import os
import numpy as np
directory = '/kaggle/working/train'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)

1101

In [4]:
ad = [i for i in files if "AD" in i]
cn = [i for i in files if "CN" in i]
emci = [i for i in files if "EMCI" in i]
lmci = [i for i in files if "LMCI" in i]
mci = [i for i in files if " MCI " in i]
print(len(ad), len(cn), len(emci), len(lmci), len(mci))

145 493 204 61 198


In [5]:
import pandas as pd
l = [[ad, 0], [cn, 1], [emci, 2], [lmci, 3], [mci, 4]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/kaggle/working/train", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])


In [6]:
images = []
for i in df["Path"]:
    try:
        images.append(np.load(i))
    except:
        print(i)
images = np.array(images)

In [7]:
import pandas as pd
df1 = pd.read_csv("/kaggle/input/handwriting-data-to-detect-alzheimers-disease/data.csv")
df1["class"] = pd.Series([1 if i == 'P' else 0 for i in df1["class"]])
df1

,ID,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,...,mean_jerk_in_air25,mean_jerk_on_paper25,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25,class
0,id_1,5160,0.000013,120.804174,86.853334,957,6601,0.361800,0.217459,103.828754,...,0.141434,0.024471,5.596487,3.184589,71,40120,1749.278166,296102.7676,144605,1
1,id_2,51980,0.000016,115.318238,83.448681,1694,6998,0.272513,0.144880,99.383459,...,0.049663,0.018368,1.665973,0.950249,129,126700,1504.768272,278744.2850,298640,1
2,id_3,2600,0.000010,229.933997,172.761858,2333,5802,0.387020,0.181342,201.347928,...,0.178194,0.017174,4.000781,2.392521,74,45480,1431.443492,144411.7055,79025,1
3,id_4,2130,0.000010,369.403342,183.193104,1756,8159,0.556879,0.164502,276.298223,...,0.113905,0.019860,4.206746,1.613522,123,67945,1465.843329,230184.7154,181220,1
4,id_5,2310,0.000007,257.997131,111.275889,987,4732,0.266077,0.145104,184.636510,...,0.121782,0.020872,3.319036,1.680629,92,37285,1841.702561,158290.0255,72575,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,id_170,2930,0.000010,241.736477,176.115957,1839,6439,0.253347,0.174663,208.926217,...,0.119152,0.020909,4.508709,2.233198,96,44545,1798.923336,247448.3108,80335,0
170,id_171,2140,0.000009,274.728964,234.495802,2053,8487,0.225537,0.174920,254.612383,...,0.174495,0.017640,4.685573,2.806888,84,37560,1725.619941,160664.6464,345835,0
171,id_172,3830,0.000008,151.536989,171.104693,1287,7352,0.165480,0.161058,161.320841,...,0.114472,0.017194,3.493815,2.510601,88,51675,1915.573488,128727.1241,83445,0
172,id_173,1760,0.000008,289.518195,196.411138,1674,6946,0.518937,0.202613,242.964666,...,0.114472,0.017194,3.493815,2.510601,88,51675,1915.573488,128727.1241,83445,0


In [8]:
import random

x = df1.sample(50)
x["class"].value_counts()

class
0    27
1    23
Name: count, dtype: int64

In [9]:
nonAlz = x[x["class"] == 0]
d = []
e = []
for i in range(len(df)):
    if df["Class"][i] == 1:
        d.append(df["Path"][i])
        e.append(df["Class"][i])

nonAlz.loc[:, "path"] = pd.Series(d)
nonAlz.loc[:, "mri-class"] = pd.Series(e)
nonAlz

/tmp/ipykernel_23/4233089575.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonAlz.loc[:, "path"] = pd.Series(d)
/tmp/ipykernel_23/4233089575.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonAlz.loc[:, "mri-class"] = pd.Series(e)


,ID,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,...,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25,class,path,mri-class
106,id_107,2380,0.000007,150.688702,170.070042,1872,4892,0.408521,0.162480,160.379372,...,3.431968,2.628899,106,46350,1561.181230,292377.27850,97400,0,/kaggle/working/train/Final CN JPEG471.npy,1
104,id_105,400,0.000005,626.609140,865.210522,5104,8977,0.690558,0.627350,745.909831,...,5.456277,2.693005,69,41645,1425.089326,165224.08860,66590,0,/kaggle/working/train/Final CN JPEG266.npy,1
151,id_152,6690,0.000012,194.082327,245.322474,2289,9325,0.446294,0.180476,219.702401,...,4.583994,3.178104,79,38095,1775.257777,192209.31800,68875,0,/kaggle/working/train/Final CN JPEG210.npy,1
159,id_160,2725,0.000010,511.533937,222.304863,1446,8956,0.852475,0.096631,366.919400,...,3.929962,1.955226,102,54065,1355.297975,178774.25360,102770,0,/kaggle/working/train/Final CN JPEG234.npy,1
164,id_165,2410,0.000014,325.502346,185.715475,2221,10056,0.418182,0.139988,255.608911,...,2.612125,2.112848,115,44255,1110.545136,184100.17520,125525,0,/kaggle/working/train/Final CN JPEG473.npy,1
95,id_96,2175,0.000010,188.083515,122.606139,1421,7488,0.221660,0.126281,155.344827,...,10.061253,5.602909,64,22425,1942.814939,101171.61210,36915,0,/kaggle/working/train/Final CN JPEG398.npy,1
102,id_103,1350,0.000010,412.184223,242.281830,2536,6504,0.186999,0.198020,327.233027,...,5.891331,3.679741,96,36985,1523.410166,156158.38820,63100,0,/kaggle/working/train/Final CN JPEG287.npy,1
153,id_154,1940,0.000006,805.235424,489.955717,3920,5620,2.322357,0.363426,647.595570,...,6.026267,4.686561,100,35855,1753.283364,174096.86020,71825,0,/kaggle/working/train/Final CN JPEG346.npy,1
142,id_143,1800,0.000008,254.998848,169.725972,1476,7247,0.203489,0.153118,212.362410,...,3.220606,2.324440,103,28280,1674.497878,224182.68560,61885,0,/kaggle/working/train/Final CN JPEG490.npy,1
170,id_171,2140,0.000009,274.728964,234.495802,2053,8487,0.225537,0.174920,254.612383,...,4.685573,2.806888,84,37560,1725.619941,160664.64640,345835,0,/kaggle/working/train/Final CN JPEG327.npy,1


In [10]:
Alz = x[x["class"] == 1]
d = []
e = []
for i in range(len(df)):
    if df["Class"][i] != 1:
        d.append(df["Path"][i])
        e.append(df["Class"][i])

Alz.loc[:, "path"] = pd.Series(d)
Alz.loc[:, "mri-class"] = pd.Series(e)

Alz

/tmp/ipykernel_23/1587798616.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Alz.loc[:, "path"] = pd.Series(d)
/tmp/ipykernel_23/1587798616.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Alz.loc[:, "mri-class"] = pd.Series(e)


,ID,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,...,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25,class,path,mri-class
76,id_77,2930,0.000016,333.839450,156.727564,2428,9491,0.191115,0.106993,245.283507,...,3.738211,3.079017,53,34515,1961.164856,93186.32455,85895,1,/kaggle/working/train/Final AD JPEG52.npy,0
57,id_58,3940,0.000012,141.251407,80.975585,1352,6373,0.208305,0.128361,111.113496,...,6.793587,2.912826,68,36550,1821.853215,142406.02100,53725,1,/kaggle/working/train/Final AD JPEG95.npy,0
20,id_21,3525,0.000010,126.323599,139.596243,1046,6118,0.225717,0.161174,132.959921,...,2.829852,2.596890,95,57310,1690.478712,159928.45270,157135,1,/kaggle/working/train/Final AD JPEG25.npy,0
23,id_24,4695,0.000007,170.640009,141.200539,1715,5983,0.182334,0.144740,155.920274,...,3.679154,2.357004,85,33965,1578.746651,201469.52850,59430,1,/kaggle/working/train/Final AD JPEG85.npy,0
13,id_14,1190,0.000008,348.475752,197.691413,1739,7297,0.189114,0.158889,273.083583,...,1.862117,3.123309,74,47020,1563.536368,224006.63110,96260,1,/kaggle/working/train/Final AD JPEG100.npy,0
18,id_19,6365,0.000011,78.339161,118.660574,1425,7755,0.127188,0.119886,98.499867,...,3.498404,2.073031,119,59035,474.049462,26984.92666,177155,1,/kaggle/working/train/Final AD JPEG38.npy,0
0,id_1,5160,0.000013,120.804174,86.853334,957,6601,0.361800,0.217459,103.828754,...,5.596487,3.184589,71,40120,1749.278166,296102.76760,144605,1,/kaggle/working/train/Final AD JPEG140.npy,0
47,id_48,4450,0.000009,105.993634,85.215193,1157,6167,0.186082,0.163422,95.604413,...,4.308988,2.928210,100,41865,1462.454915,182825.98810,92150,1,/kaggle/working/train/Final AD JPEG83.npy,0
26,id_27,3365,0.000006,139.705985,179.776831,1777,6415,0.206496,0.208559,159.741408,...,1.809954,2.213869,180,74185,1632.469098,352981.85000,238895,1,/kaggle/working/train/Final AD JPEG64.npy,0
11,id_12,1970,0.000011,231.499777,90.649480,1434,5643,0.209560,0.144054,161.074629,...,2.022657,1.528793,104,71795,912.940525,145022.11890,182355,1,/kaggle/working/train/Final AD JPEG47.npy,0


In [11]:
nonAlz_y_handwriting = np.zeros(nonAlz.shape[0])
nonAlz_y_mri = nonAlz["mri-class"]

Alz_y_handwriting = np.ones(Alz.shape[0])
Alz_y_mri = Alz["mri-class"]

In [12]:
import pickle

with open('/kaggle/input/handwriitng-classifier/other/claassifier-gb/1/gb_classifier_weights.pkl', 'rb') as file:
    handwriting_model = pickle.load(file)

In [13]:
import random
s = [nonAlz, Alz]
x = []
for i in range(10):
    r = random.choice(s)
    x.append(r.sample(1))

In [14]:
def predictor(data):
    inp = data.drop(["ID", "class", "path", "mri-class"], axis=1)
    handwriting_prediction = handwriting_model.predict(inp)
    if handwriting_prediction.any() == 1:
        img = np.load(data["path"].values[0])
        img = np.expand_dims(img, axis=0) 
        prediction = np.argmax(model.predict(img), axis=1)[0]
        print(f"Model predicted - You have alzheimer's, type {prediction}")
        print(f"Actual case - You have alzheimer's, type {data['mri-class'].values[0]}")
    else:
        print("Model - predicted - You dont have alzheimer's")
        print(f"Actual case - You {'have' if data['class'].any() == 1 else 'dont have'} alzheimer's")

In [15]:
for i in x:
    predictor(i)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Model predicted - You have alzheimer's, type 0
Actual case - You have alzheimer's, type 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Model predicted - You have alzheimer's, type 0
Actual case - You have alzheimer's, type 0
Model - predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
Model - predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
Model - predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


I0000 00:00:1713924013.249040      70 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Model predicted - You have alzheimer's, type 1
Actual case - You have alzheimer's, type 0
Model - predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
Model - predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
Model - predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
Model - predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
